## Этап 1: загрузка и предобработка данных

In [1]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Вывод

1. Обратил внимание на наличие нулевых значений в столбцах days_employed и total_income. Эти столбцы имеют числовой вид, поэтому пропущенные данные являются NaN

In [2]:
data.isnull().sum() #узнал, сколько пропусков
median_days_employed = data['days_employed'].median() #медиана дней занятости
mean_days_employed = data['days_employed'].mean() #средний период занятости - даёт отрицательное значение

mean_income = data['total_income'].mean() #посмотрел средний доход
median_income = data['total_income'].median() #посмотрел медиану дохода

data['days_employed'] = data['days_employed'].fillna(mean_days_employed) #заполнил параметром средней занятости из-за того, что медиана даёт отрицательное значение

data['total_income'].unique 
data['total_income'] = data['total_income'].fillna(median_income)#заполнил пропуски медианой дохода
data.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Вывод

1. Найдено равное количество NaN в столбцах days_employed и total_income
2. Если у клиента NaN в days_employed, то у того же клиента NaN в total_income.
3. Возможно, банку не известно об уровне дохода или занятости клиента.
4. NaN в days_employed заполнен средним значение по всему столбцу. Отказался от медианы, так как она показывает отрицательное значение.
5. NaN в total_income заполнен медианой по выборке.


In [3]:
data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


### Вывод

Общая информация по info() показала, что вещественный тип данных был в столбцах days_employed и total_income. После применения astype('int') через присваивание, тип данных был изменён на целочисленный.

### Обработка дубликатов

In [4]:
data['education'] = data['education'].str.lower()
data['family_status'] = data['family_status'].str.lower()
data['gender'] = data['gender'].str.lower()
data['income_type'] = data['income_type'].str.lower()
data['purpose'] = data['purpose'].str.lower()


data['education'].value_counts() 
data['family_status'].value_counts() 
data['gender'].value_counts() 
data['income_type'].value_counts()
data['purpose'].value_counts()


свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
покупка жилья для сдачи                   653
операции с жильем                         653
операции с коммерческой недвижимостью     651
покупка жилья                             647
жилье                                     647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
строительство недвижимости                620
покупка своего жилья                      620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

### Вывод

1. поиск дубликатов осуществлялся в объектных столбцах.
2. в gender помимо f и m, вошёл третий пол xna в единственном случае
3. приведение строк к нижнему регистру позволило справиться с проблемой дубликатов во всех столбцах, за исключением purpose. Люди (менеджеры) описывали цели кредитования по-разному. Здесь сыграл свою роль человеческий фактор, поэтому необходимо прибегнуть к лемматизации.
4. Дубликаты возникали из-за человеческого фактора

## Этап 2: Лемматизация для выделения категорий целей кредита

In [5]:
from pymystem3 import Mystem
m = Mystem()
from collections import Counter
from nltk.stem import SnowballStemmer 
russian_stemmer = SnowballStemmer('russian')

lst = [i for i in data['purpose']]
lemms = []
for i in lst: #Сенсей, простите за "ВЦ"код
   lemms+=m.lemmatize(i)

print(Counter(lemms)) 

dict_lemm = ['жилая','коммерческая','недвижимость', 'недвижимость', 'автомобиль', 'образование', 'свадьба', 'ремонт']

stem_lst = [] #в попытках найти способ подбора "идеального" способа оформить единообразие в целях провёл стемминг слов
#из слов после лемматизации.
for x in lemms:
    stem_lst.append(russian_stemmer.stem(x))
    
print(Counter(stem_lst))


'''

Лемматизация помогла из данных выявить основные категории целей кредита: свадьба, автомобиль, недвижимость, 
образование, ремонт. После этого предстояло придумать инстурмент, который будет анализировать старый вариант
описания цели и заменять его на унифицированный.


'''


purposes_dict = {'свадьба': ['свадьба', 'на проведение свадьбы', 'сыграть свадьбу'],
'автомобиль': ['на покупку своего автомобиля','автомобиль','сделка с подержанным автомобилем', 'свой автомобиль',
'на покупку подержанного автомобиля', 'автомобили', 'на покупку автомобиля','приобретение автомобиля',
'сделка с автомобилем'],
'недвижимость': ['операции с недвижимостью', 'покупка коммерческой недвижимости',
'покупка жилья для сдачи','операции с жильем', 'операции с коммерческой недвижимостью',
'жилье', 'покупка жилья', 'покупка жилья для семьи', 'строительство собственной недвижимости',
'недвижимость', 'операции со своей недвижимостью', 'строительство жилой недвижимости', 'покупка недвижимости',
'строительство недвижимости', 'покупка своего жилья', 'покупка жилой недвижимости'], 
                 'образование': ['заняться высшим образованием','дополнительное образование', 'высшее образование',
                                'получение дополнительного образования', 'образование', 'получение образования',
                                'профильное образование', 'получение высшего образования', 'заняться образованием'],
                 'ремонт': ['ремонт жилью']}


   
def change_purpose(data, dict):
    temp_lst = list(data['purpose']) #создаём списко из значений колонки с целями
    new_purposes_list = [] #пустой список, который будет заполнен в цикле
    new_purposes_dict = {'new_purpose':new_purposes_list} #создаваемый словарь с ключём new_purpose для входа в общую таблицу

    for s in temp_lst: #начинаем перебор старых целей
        for key, value in dict.items(): #включаем в работу перебор ключей и значений
            for u in value: #под микроскопом всматриваемся в значения 
                if s == u: 
                    new_purposes_list.append(key) #добавление в список ключ из водимого 
                    #словаря, если нашлось совпадение между стройкой ячейки и значением одного из ключей
              
    new_table = pd.DataFrame(new_purposes_list) #создайм новый датафрейм из словаря, который в цикде уже был заполнен значениями.
    data['new_purpose'] = new_table #легко и просто в столбец добавили новый датафрейм. Старый значения оставил специально.
    return

change_purpose(data, purposes_dict) 
    
data.head(10)

Counter({' ': 33677, '\n': 21525, 'недвижимость': 6367, 'покупка': 5912, 'жилье': 4473, 'автомобиль': 4315, 'образование': 4022, 'с': 2924, 'операция': 2610, 'свадьба': 2348, 'свой': 2235, 'на': 2233, 'строительство': 1881, 'высокий': 1375, 'получение': 1316, 'коммерческий': 1315, 'для': 1294, 'жилой': 1233, 'сделка': 944, 'дополнительный': 909, 'заниматься': 908, 'проведение': 777, 'сыграть': 774, 'сдача': 653, 'семья': 641, 'собственный': 635, 'со': 630, 'ремонт': 612, 'подержанный': 489, 'подержать': 479, 'приобретение': 462, 'профильный': 436})
Counter({' ': 33677, '\n': 21525, 'недвижим': 6367, 'покупк': 5912, 'жил': 5706, 'автомобил': 4315, 'образован': 4022, 'с': 2924, 'операц': 2610, 'свадьб': 2348, 'сво': 2235, 'на': 2233, 'строительств': 1881, 'высок': 1375, 'получен': 1316, 'коммерческ': 1315, 'для': 1294, 'подержа': 968, 'сделк': 944, 'дополнительн': 909, 'занима': 908, 'проведен': 777, 'сыгра': 774, 'сдач': 653, 'сем': 641, 'собствен': 635, 'со': 630, 'ремонт': 612, 'приоб

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,new_purpose
0,1,-8437,42,высшее,0,женат / замужем,0,f,сотрудник,0,253875,покупка жилья,недвижимость
1,1,-4024,36,среднее,1,женат / замужем,0,f,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,-5623,33,среднее,1,женат / замужем,0,m,сотрудник,0,145885,покупка жилья,недвижимость
3,3,-4124,32,среднее,1,женат / замужем,0,m,сотрудник,0,267628,дополнительное образование,образование
4,0,340266,53,среднее,1,гражданский брак,1,f,пенсионер,0,158616,сыграть свадьбу,свадьба
5,0,-926,27,высшее,0,гражданский брак,1,m,компаньон,0,255763,покупка жилья,недвижимость
6,0,-2879,43,высшее,0,женат / замужем,0,f,компаньон,0,240525,операции с жильем,недвижимость
7,0,-152,50,среднее,1,женат / замужем,0,m,сотрудник,0,135823,образование,образование
8,2,-6929,35,высшее,0,гражданский брак,1,f,сотрудник,0,95856,на проведение свадьбы,свадьба
9,0,-2188,41,среднее,1,женат / замужем,0,m,сотрудник,0,144425,покупка жилья для семьи,недвижимость


### Вывод

Проведённая лемматизация позволила сузиться на основных категориях: свадьба, автомобиль, недвижимость, 
образование, ремонт.<br> 
Стемминг может быть полезнее лемматизации, если речь идёт о более сложном сравнении данных. <br> 
В датафрейм нельзя добавлять просто список. Для этого нужно создать словарь. <br> 
<br> 
<br> 

## Этап 3: Категоризация данных   
Провожу анализ взаимосвязей:  
1) Наличие детей у заёмщика и возврат займа в срок.
2) Семейное положение и возврат займа в срок.
3) Уровень дохода и возврат кредита в срок.
4) Цель кредита и его возврат в срок

In [1]:
children_debt = data[['children','debt']] 
debt_grouped = children_debt.groupby('children').sum() 

parents_debt = debt_grouped.loc[1:, 'debt'].sum() 
childless_debt = debt_grouped.loc[0, 'debt'].sum()
total_debt = parents_debt + childless_debt 

percent_total_debt = total_debt/len(data)

percent_parents_debt = parents_debt / total_debt 
percent_childless_debt = childless_debt / total_debt 

family_data = data[['family_status', 'debt']]
new_family_data = family_data.groupby('family_status').sum()

col = ['family_status', 'count']
notice_count = family_data['family_status'].value_counts() 
pd.DataFrame(data = notice_count, columns = col)
notice_count = notice_count.reset_index() 
notice_count.set_axis(['family_status', 'count'],axis = 'columns',inplace = True) 

common_table = new_family_data.merge(notice_count, on='family_status', how='left')
common_table
common_table['ratio'] = common_table.loc[:,'debt']/common_table.loc[:,'count']

income_debt = data[['total_income','debt']]
income_debt = income_debt.sort_values('total_income', ascending = False)
median_income = int(income_debt['total_income'].median())
above_median = int(median_income + (median_income/4))
below_median = int(median_income - (median_income/4))


income_debt['income_groups'] = income_debt['total_income'].astype('str')

def income_categories (income):
    try:
        if above_median > income > median_income:
            return 'выше медианы'
        elif income >= above_median:
            return 'высокий доход'
        elif median_income > income >= below_median:
            return'ниже медианы'
        elif below_median >= income:
            return 'низкий доход'
    except:
        return 'Error'
    return
     
income_debt['income_groups'] = income_debt['total_income'].apply(income_categories)

result_income_debt = income_debt[['income_groups', 'debt']].groupby('income_groups').count()

purposes_debt = data[['new_purpose','debt']]


number_categories = purposes_debt['new_purpose'].value_counts()
purposes_debt = purposes_debt.groupby('new_purpose').sum()
purposes_debt['number'] = number_categories
purposes_debt['ratio'] = purposes_debt['debt']/purposes_debt['number']


NameError: name 'data' is not defined

### Вывод

Категоризация данных произведена следующим образом:<br> 
<br> 
Для анализ взаимосвязи наличия детей у заёмщика и возвратом займа в срок в качестве исходных данных выбраны столбцы **children, debt**. Мне нужно было сопоставить факт наличия ребёнка со случаями невозврата кредита. Поэтому я посчитал общее количество невозвратов в % и распределил это количество на бездетных и клиентов с детьми.<br> 
<br> 
Для анализ взаимосвязи семейного положения и возвратом займа в срок в качестве исходных данных выбраны столбцы **family_status, debt**. По каждому семейному положению посчитал отношение количества случаев просрочки этой группы на общее количество записей этой группы.<br> 
<br> 
Для анализ взаимосвязи между уровнем дохода и возвратом кредита в срок в качестве исходных данных выбраны столбцы **total_income, debt**. Колонка с доходомо нужна была для понимания распределения уровня доходов. Были определены 4 категории суммы дохода. Каждая категория получила расчёт количества случае просрочки.<br> 
<br> 
Для анализ взаимосвязи цели кредита и его возвратом в срок в качестве исходных данных выбраны столбцы **new_purpose, debt**. После лемматизации были выявлены 5 основных групп целей. Для каждой категории было выявлено количество просрочек.

### Исслоедование зависимостей

In [7]:
print('Всего клиентов с просрочками: {:%}'.format(percent_total_debt))
print('Из них клиентов с детьми: {:%}'.format(percent_parents_debt))
print('Из них бездетных клиентов: {:%}'.format(percent_childless_debt))
print('Разница в {:.2} раза'.format(percent_childless_debt/percent_parents_debt))

Всего клиентов с просрочками: 8.083624%
Из них клиентов с детьми: 38.908046%
Из них бездетных клиентов: 61.091954%
Разница в 1.6 раза


### Вывод

Бездетные клиенты оказались более неблагонадёжными заёмщиками, нежели клиенты с детьми.


In [8]:
common_table.sort_values(by = 'ratio', ascending = False)


,family_status,debt,count,ratio
4,не женат / не замужем,274,2813,0.097405
2,гражданский брак,388,4177,0.092890
3,женат / замужем,931,12380,0.075202
0,в разводе,85,1195,0.071130
1,вдовец / вдова,63,960,0.065625


### Вывод

**Самыми неблагонажёными** заёмщиками оказались люди без офицально закреплённых отношений со статусами: <br> 
не женат / не замужем и гражданский брак. Вероятно, эти категории могут быть объеденены в одну.<br> 
<br> 
Имеющие или имевшие статус нахождения в официальном браке заняли второе место.<br> 

*Удивительный факт: самыми благонадёжными заёмщиками оказались вдовцы и вдовы*




In [9]:
result_income_debt

,debt
income_groups,
высокий доход,6338
выше медианы,3337
ниже медианы,4174
низкий доход,5501


### Вывод

Самыми благонадёжными заёмщиками показали люди с уровнем дохода, приближенным к медиане по выборе.<br> 
"Хвосты" распределения показали более негативную динамику.<br> 
Самыми неблагонадёжными оказались люди с наибольшим доходом.<br> 
Отметим, что лучший результа достался с клиентами, которые имеют доход выше медианы, но не вошли в список самых состоятельных.<br> 
<br> 
**Можно сделать вывод о том, что лучшим заёмщиком оказался представитель среднего класса.**

In [10]:
purposes_debt

,debt,number,ratio
new_purpose,,,
автомобиль,403,4315,0.093395
недвижимость,747,10228,0.073035
образование,370,4022,0.091994
ремонт,35,612,0.057190
свадьба,186,2348,0.079216


### Вывод

Чаще всего просрочка вознекает по кредиту, взятому на покупку автомобиля или получение образования.<br> 
"Свадебные" кредиты и займы с целью покупки недвижимости имею практически равный процент случаев просрочки.<br> 
Самыми благонадёжными оказались желающие сделать дома ремонт, однако выборка по данной категории оказалсь самой небольшой.


### Общий вывод

Анализ позволил разобраться в 4 факторах, которые могут стать базой для определения степени благонадёжности клиента:
<br> 1) цель кредита
<br> 2) семейное положение
<br> 3) категория уровня дохода
<br> 4) наличие детей

Например, на основе полученны данных можно сформировать портрет идеального заёмщика:
<br> 1) Это женатый человек (или вдовец/вдова) с детьми
<br> 2) С достатком выше медианы, но не входящий в группу с высоким доходом.
<br> 3) Цель кредита: ремонт.
